# Bi-LSTM on TREC-6

## Librairies

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../.."))

In [2]:
import time
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorWithPadding

from esntorch.utils.embedding import EmbeddingModel

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Global variables

In [4]:
MODEL_FILE =   '/raid/home/jeremiec/Ax_results/Bi-LSTM/trec-6_bilstm-128_10ep_meanpooling_model.pt'
RESULTS_FILE = '/raid/home/jeremiec/Ax_results/Bi-LSTM/trec-6_bilstm-128_10ep_meanpooling_results.pkl'
CACHE_DIR = '/raid/home/jeremiec/huggingface_datasets' # path of your folder

## Dataset

In [5]:
# rename correct column as 'labels': depends on the dataset you load

def tokenize(sample):
    """Tokenize sample"""
    
    sample = tokenizer(sample['text'], truncation=True, padding=False, return_length=True)
    
    return sample
    
def load_and_enrich_dataset(dataset_name, split, cache_dir):
    """
    Load dataset from the datasets library of HuggingFace.
    Tokenize and add length.
    """
    
    # Load dataset
    dataset = load_dataset(dataset_name, split=split, cache_dir=CACHE_DIR)
    
    # Rename label column for tokenization purposes (use 'label-fine' for fine-grained labels)
    dataset = dataset.rename_column('label-coarse', 'labels')
    
    # Tokenize data
    dataset = dataset.map(tokenize, batched=True)
    dataset = dataset.rename_column('length', 'lengths')
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'lengths'])
    
    return dataset

In [6]:
CACHE_DIR = '/raid/home/jeremiec/huggingface_datasets'

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

full_train_dataset = load_and_enrich_dataset('trec', split='train', cache_dir=CACHE_DIR).sort("lengths")
train_val_datasets = full_train_dataset.train_test_split(train_size=0.8, shuffle=True)
train_dataset = train_val_datasets['train'].sort("lengths")
val_dataset = train_val_datasets['test'].sort("lengths")

test_dataset = load_and_enrich_dataset('trec', split='test', cache_dir=CACHE_DIR).sort("lengths")

dataset_d = {
    'full_train': full_train_dataset,
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
    }

dataloader_d = {}
for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

Using custom data configuration default
Reusing dataset trec (/raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)
Loading cached processed dataset at /raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9/cache-848d2c8f39949357.arrow
Loading cached sorted indices for dataset at /raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9/cache-b41d7661aa7925e9.arrow
Loading cached split indices for dataset at /raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9/cache-9441de5c9d7aee43.arrow and /raid/home/jeremiec/huggingface_datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9/cache-6ef7be09f24a2150.arrow
Loading cached sorted indices for dataset at /raid/home/jeremiec/hu

In [7]:
dataset_d

{'full_train': Dataset({
     features: ['attention_mask', 'input_ids', 'label-fine', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 5452
 }),
 'train': Dataset({
     features: ['attention_mask', 'input_ids', 'label-fine', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 4361
 }),
 'val': Dataset({
     features: ['attention_mask', 'input_ids', 'label-fine', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 1091
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'label-fine', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 500
 })}

## Model

In [9]:
class RNN(nn.Module):
    
    def __init__(self, embedding_dim,
                 hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        super().__init__()
        
        self.embedding = EmbeddingModel('bert-base-uncased', device)
        
        # recurrent hidden layers (LSTM)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        # fully connected output layer (fc)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, batch):
        
        packed_embedded = self.dropout(self.embedding.get_embedding(batch))        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [10]:
output_dim = len(set(full_train_dataset['labels'].tolist()))

In [11]:
output_dim

6

In [12]:
# Parameters

# INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 768
HIDDEN_DIM = 128 # 256 was default
OUTPUT_DIM = output_dim # 1 (binary) or output_dim (*** multiclass ***)
N_LAYERS = 1 # 2 was default
BIDIRECTIONAL = True
DROPOUT = 0.5
# PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [13]:
# Model

model = RNN(EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


/opt/conda/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [14]:
def count_parameters(model):
    
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 921,094 trainable parameters


## Training

In [15]:
# Adam optimizer
optimizer = optim.Adam(model.parameters())

In [16]:
# Criterion
# criterion = nn.BCEWithLogitsLoss() # binary
criterion = nn.CrossEntropyLoss() # *** multiclass ***

In [17]:
# Put model and criterion to device
model = model.to(device)
criterion = criterion.to(device)

In [18]:
# binary accuracy
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds)) # thresholding output neuron
    correct = (rounded_preds == y).float()            # convert into float for division 
    acc = correct.sum() / len(correct)
    
    return acc

In [19]:
# *** multiclass ***
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    
    return correct.sum().item() / torch.FloatTensor([y.shape[0]])

In [20]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions_raw = model(batch)
                        
        predictions = predictions_raw.squeeze(1)
                
        loss = criterion(predictions, batch['labels'])
        
        # acc = binary_accuracy(predictions, batch.label)        # *** binary *** 
        acc = categorical_accuracy(predictions, batch['labels']) # *** multiclass ***
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch).squeeze(1)
            
            loss = criterion(predictions, batch['labels'])
            
            # acc = binary_accuracy(predictions, batch.label)        # *** binary ***
            acc = categorical_accuracy(predictions, batch['labels']) # *** multiclass ***

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
import time

def epoch_time(start_time, end_time):
    
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 10 # 30

best_valid_loss = float('inf')

t0 = time.time()

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, dataloader_d['train'], optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, dataloader_d['val'], criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_FILE)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

t1 = time.time()
train_time = t1 - t0
print('training time: {}'.format(train_time))

hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128])
hidden 2: torch.Size([256, 256])
hidden 1: torch.Size([2, 256, 128]

KeyboardInterrupt: 

In [25]:
model = RNN(EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT)
model.load_state_dict(torch.load(MODEL_FILE))
model.eval().to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RNN(
  (rnn): LSTM(768, 128, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=256, out_features=6, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [26]:
test_loss, test_acc = evaluate(model, dataloader_d['test'], criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

results = test_acc, train_time, count_parameters(model)

Test Loss: 0.246 | Test Acc: 92.35%


In [27]:
results

(0.9234758913516998, 102.64727020263672, 921094)

In [28]:
with open(RESULTS_FILE, 'wb') as fh:
    pickle.dump(results, fh)

In [8]:
with open(RESULTS_FILE, 'rb') as fh:
    res = pickle.load(fh)

In [9]:
res

(0.9295273721218109, 97.01584601402283, 921094)